In [76]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd

# 1. Read Data

In [77]:
path = "~/Desktop/kaggle/Porto/"

In [78]:
df = pd.read_csv(path+'train.csv',index_col='id')#,nrows=50000)

In [79]:
df = df.reset_index(drop=True)

In [80]:
df.head()

,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,0,2,2,5,1,0,0,1,0,0,...,9,1,5,8,0,1,1,0,0,1
1,0,1,1,7,0,0,0,0,1,0,...,3,1,1,9,0,1,1,0,1,0
2,0,5,4,9,1,0,0,0,1,0,...,4,2,7,7,0,1,1,0,1,0
3,0,0,1,2,0,0,1,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,0,0,2,0,1,0,1,0,0,0,...,3,1,1,3,0,0,0,1,1,0


# 2. Get Features and Target

In [81]:
features = list(df.columns)
target = 'target'
features.remove(target)

In [82]:
X = np.array(df[features])
y = df[target]

# 5. Build Model

In [83]:
model = Sequential()
model.add(Dense(25, input_dim=57, activation='relu'))
model.add(Dense(57, activation='relu'))

In [89]:
from keras import optimizers

optimizers.Adam(lr=0.5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(X, 
          X, 
          epochs=20, 
          batch_size=len(X), 
          #validation_split=0.2, 
          verbose=1)

Epoch 1/20
595212/595212 [==============================] - 4s - loss: 0.6444
Epoch 2/20
595212/595212 [==============================] - 3s - loss: 0.6920
Epoch 3/20
595212/595212 [==============================] - 5s - loss: 0.6437
Epoch 4/20
595212/595212 [==============================] - 3s - loss: 0.6518
Epoch 5/20
595212/595212 [==============================] - 3s - loss: 0.6648
Epoch 6/20
595212/595212 [==============================] - 3s - loss: 0.6514
Epoch 7/20
595212/595212 [==============================] - 4s - loss: 0.6367
Epoch 8/20
595212/595212 [==============================] - 3s - loss: 0.6361
Epoch 9/20
595212/595212 [==============================] - 4s - loss: 0.6426
Epoch 10/20
595212/595212 [==============================] - 3s - loss: 0.6430
Epoch 11/20
595212/595212 [==============================] - 3s - loss: 0.6359
Epoch 12/20
595212/595212 [==============================] - 3s - loss: 0.6284
Epoch 13/20
595212/595212 [==============================] - 

In [90]:
len(model.layers[0].get_weights())

2

In [91]:
model2 = Sequential()
model2.add(Dense(25, input_dim=57, weights=model.layers[0].get_weights()))
#model2.add(Activation('relu'))
#model2.add(Activation('relu'))

new_X = model2.predict(X)

In [92]:
new_X.shape

(595212, 25)

In [93]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
 
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_coefficient(preds,dtrain):
    y = dtrain.get_label()
    return 'gini', -gini_normalized(y,preds)

In [94]:
from sklearn.model_selection import KFold

from xgboost import XGBClassifier
xgb = XGBClassifier(max_depth=4, 
                    learning_rate=0.04, 
                    n_estimators=1000, 
                    objective='binary:logistic',
                    nthread=-1, 
                    gamma=0, 
                    subsample=0.8,
                    colsample_bytree=0.8,  
                    scale_pos_weight=30, 
                    missing=None)

In [95]:
kf = KFold(n_splits=2)
for train_index, test_index in kf.split(X):
    train_X, test_X = X[train_index], X[test_index]
    train_y, test_y = y[train_index], y[test_index]
    xgb.fit(train_X, train_y, 
            eval_set=[(train_X,train_y),(test_X,test_y)], 
            eval_metric=gini_coefficient,
            early_stopping_rounds=10,
            verbose=10)

[0]	validation_0-error:0.580019	validation_1-error:0.582774	validation_0-gini:-0.198663	validation_1-gini:-0.190952
Multiple eval metrics have been passed: 'validation_1-gini' will be used for early stopping.

Will train until validation_1-gini hasn't improved in 10 rounds.
[10]	validation_0-error:0.562667	validation_1-error:0.564871	validation_0-gini:-0.253134	validation_1-gini:-0.234274
[20]	validation_0-error:0.552012	validation_1-error:0.554011	validation_0-gini:-0.265306	validation_1-gini:-0.244614
[30]	validation_0-error:0.537214	validation_1-error:0.538386	validation_0-gini:-0.273344	validation_1-gini:-0.250521
[40]	validation_0-error:0.526421	validation_1-error:0.527483	validation_0-gini:-0.27947	validation_1-gini:-0.254332
[50]	validation_0-error:0.521159	validation_1-error:0.522587	validation_0-gini:-0.285288	validation_1-gini:-0.257831
[60]	validation_0-error:0.513313	validation_1-error:0.514691	validation_0-gini:-0.290184	validation_1-gini:-0.259926
[70]	validation_0-error:

In [97]:
kf = KFold(n_splits=2)
for train_index, test_index in kf.split(new_X):
    train_X, test_X = new_X[train_index], new_X[test_index]
    train_y, test_y = y[train_index], y[test_index]
    xgb.fit(train_X, train_y, 
            eval_set=[(train_X,train_y),(test_X,test_y)], 
            eval_metric=gini_coefficient,
            early_stopping_rounds=10,
            verbose=10)

[0]	validation_0-error:0.758083	validation_1-error:0.758792	validation_0-gini:-0.108212	validation_1-gini:-0.090112
Multiple eval metrics have been passed: 'validation_1-gini' will be used for early stopping.

Will train until validation_1-gini hasn't improved in 10 rounds.
[10]	validation_0-error:0.714162	validation_1-error:0.714814	validation_0-gini:-0.169482	validation_1-gini:-0.126536
[20]	validation_0-error:0.698598	validation_1-error:0.700312	validation_0-gini:-0.180907	validation_1-gini:-0.130553
[30]	validation_0-error:0.706841	validation_1-error:0.708978	validation_0-gini:-0.194698	validation_1-gini:-0.135007
[40]	validation_0-error:0.69512	validation_1-error:0.698269	validation_0-gini:-0.211914	validation_1-gini:-0.138439
[50]	validation_0-error:0.679835	validation_1-error:0.683373	validation_0-gini:-0.224861	validation_1-gini:-0.141196
[60]	validation_0-error:0.665917	validation_1-error:0.669593	validation_0-gini:-0.235666	validation_1-gini:-0.142746
[70]	validation_0-error:

KeyboardInterrupt: 

In [ ]:
# this is your initial model
model = Sequential()
model.add(Dense(20, 64, init='uniform'))
model.add(Activation('tanh'))
model.add(Dense(64, 1, init='uniform'))
model.add(Activation('softmax'))

# we train it
model.compile(loss='mse', optimizer='sgd')
model.fit(X_train, y_train, nb_epoch=20, batch_size=16)

# we build a new model with the activations of the old model
# this model is truncated after the first layer
model2 = Sequential()
model2.add(Dense(20, 64, weights=model.layers[0].get_weights()))
model2.add(Activation('tanh'))

activations = model2._predict(X_batch)